In [15]:
import cv2
import numpy as np
import os

In [16]:
template = cv2.imread('data/test.jpg', 0)  # Load template image in grayscale
target = cv2.imread('data/sample.jpg', 0)  # Load target image in grayscale

scale_factor = 4  # Upscale by 4x
template_upscaled = cv2.resize(template, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_CUBIC)

print(f'image shape: {target.shape}')
print(f'template shape: {template.shape}')
print(f'template_upscaled shape: {template_upscaled.shape}')

image shape: (9000, 12600)
template shape: (61, 48)
template_upscaled shape: (244, 192)


In [17]:
# Initialize ORB detector
# orb = cv2.ORB_create()
sift = cv2.SIFT_create()

# Detect and compute keypoints and descriptors
keypoints_template, descriptors_template = sift.detectAndCompute(template, None)
keypoints_target, descriptors_target = sift.detectAndCompute(target, None)

In [18]:
# Use FLANN-based matcher
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)  # Adjust checks for speed/accuracy tradeoff
flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(descriptors_template, descriptors_target, k=2)

In [19]:
# Apply ratio test to filter matches
good_matches = []
for m, n in matches:
    if m.distance < 0.6 * n.distance:  # Adjust threshold as necessary
        good_matches.append(m)

good_matches

[< cv2.DMatch 0000027DE91C0F30>, < cv2.DMatch 0000027DEE3628B0>]

In [13]:
# Draw matches on the images
match_img = cv2.drawMatches(
    template, keypoints_template,
    target, keypoints_target,
    good_matches, None,
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
)

# # Display the matched keypoints
# cv2.imshow('Matched Keypoints', match_img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [14]:
# Localization of detected region (Optional)
if len(good_matches) > 4:  # Minimum number of matches for homography
    src_pts = np.float32([keypoints_template[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints_target[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

    # Compute Homography
    H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

    # Get template dimensions
    h, w = template.shape[:2]
    corners = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1, 1, 2)

    # Transform template corners to target image space
    transformed_corners = cv2.perspectiveTransform(corners, H)

    # Draw bounding box on target image
    target_with_box = target.copy()
    target_with_box = cv2.polylines(target_with_box, [np.int32(transformed_corners)], True, (0, 255, 0), 3)

    # Create a filename based on scale and threshold
    output_folder = 'output'  # Specify your output folder
    os.makedirs(output_folder, exist_ok=True)  # Create output folder if it doesn't exist
    filename = f"detected_scale_1.jpg"
    filepath = os.path.join(output_folder, filename)

    # Save the detected image
    cv2.imwrite(filepath, target_with_box)

    # Print the count of detected items
    print(f'File Stored at: {filepath}\nfile_name: {filename}')
    # Display the target image with the bounding box
    cv2.imshow('Detected Region', target_with_box)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("Not enough good matches were found for homography.")


File Stored at: output\detected_scale_1.jpg
file_name: detected_scale_1.jpg
